# Introduction
1. Download dataset from https://www.yelp.com/dataset
2. Place the dataset (review.json and user.json) in the same directory as this notebook
3. Run notebook to obtain filtered data based on the review_stars
----------------------
# Content
Main Objective: To load review.json and user.json and convert them into csv 

This is the breakdown of the Notebook:
1. Load data in batches due to its sheer size
2. Join review and user dataset
3. Did some basic EDA by performing groupby operations
4. Filter usefulness rating > 1
5. Save dataset into respective review stars

----------------------
# Summary
We successfully preprocessed and converted the datasets to csv files where it is ready for more EDA, and subsequently, more filtering/cleaning if needed

In [1]:
import pandas as pd

In [4]:
json_file = "review.json"
size = 1000000
review = pd.read_json(json_file, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

In [11]:
# There are multiple chunks to be read
chunk_list = []
for chunk_review in review:
    # Drop columns that aren't needed
    #chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})

    # Show feedback on progress
    print(f"{chunk_review.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_review)
    
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

1000000 out of 1,000,000 related reviews
1000000 out of 1,000,000 related reviews
1000000 out of 1,000,000 related reviews
1000000 out of 1,000,000 related reviews
1000000 out of 1,000,000 related reviews
1000000 out of 1,000,000 related reviews
685900 out of 1,000,000 related reviews


In [5]:
user = pd.read_json("user.json", lines=True, chunksize=size)

In [6]:
user

In [7]:
chunk_list = []
for chunk_review in user:
    
    # Show feedback on progress
    print(f"{chunk_review.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_review)
    
# After trimming down the review file, concatenate all relevant data back to one dataframe
df2 = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

1000000 out of 1,000,000 related reviews
637138 out of 1,000,000 related reviews


In [8]:
df2.columns

Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')

In [14]:
df2.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,l6BmjZMeQD3rDxWUbiAiow,Rashmi,95,2013-10-08 23:11:33,84,17,25,"2015,2016,2017","c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g...",5,...,0,0,0,0,1,1,1,1,2,0
1,4XChL029mKr5hydo79Ljxg,Jenna,33,2013-02-21 22:29:06,48,22,16,,"kEBTgDvFX754S68FllfCaA, aB2DynOxNOJK9st2ZeGTPg...",4,...,0,0,0,0,0,0,1,1,0,0
2,bc8C_eETBWL0olvFSJJd0w,David,16,2013-10-04 00:16:10,28,8,10,,"4N-HU_T32hLENLntsNKNBg, pSY2vwWLgWfGVAAiKQzMng...",0,...,0,0,0,0,1,0,0,0,0,0
3,dD0gZpBctWGdWo9WlGuhlA,Angela,17,2014-05-22 15:57:30,30,4,14,,"RZ6wS38wnlXyj-OOdTzBxA, l5jxZh1KsgI8rMunm-GN6A...",5,...,0,0,0,0,0,2,0,0,1,0
4,MM4RJAeH6yuaN8oZDSt0RA,Nancy,361,2013-10-23 07:02:50,1114,279,665,"2015,2016,2017,2018","mbwrZ-RS76V1HoJ0bF_Geg, g64lOV39xSLRZO0aQQ6DeQ...",39,...,1,0,0,1,16,57,80,80,25,5


In [13]:
df.head(1)

,review_id,user_id,business_id,review_stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36


## Group by the review_stars and usefulness rating
purpose was to get an estimate of the distribution

In [30]:
df[df['useful']>0].groupby(['review_stars','useful']).count()

review_id  user_id  business_id   funny    cool    text  \
review_stars useful                                                            
1            1          212033   212033       212033  212033  212033  212033   
             2          117676   117676       117676  117676  117676  117676   
             3           69887    69887        69887   69887   69887   69887   
             4           44713    44713        44713   44713   44713   44713   
             5           30174    30174        30174   30174   30174   30174   
...                        ...      ...          ...     ...     ...     ...   
5            210             1        1            1       1       1       1   
             215             2        2            2       2       2       2   
             241             1        1            1       1       1       1   
             278             1        1            1       1       1       1   
             358             1        1            1       1       1       1   

                       date  
review_stars useful          
1            1       212033  
             2       117676  
             3        69887  
             4        44713  
             5        30174  
...                     ...  
5            210          1  
             215          2  
             241          1  
             278          1  
             358          1  

[839 rows x 7 columns]

In [32]:
df.head(1)

,review_id,user_id,business_id,review_stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36


# Review star 1 and 5 have sufficient amount of training data

However, review star 5 have significantly more data than review star 1. Hence, we further filtered it to have usefulness rating of 2 instead of 1. \
There might be a concern of biasness in this case. \
However, the rationale behind this was that, in a user-generated content platform, a usefulness rating of 1 vs 2 is not much of a difference as the usefulness ratings could easily go above 50.

In [34]:
df[df['useful']>1].groupby('review_stars').count()

,review_id,user_id,business_id,useful,funny,cool,text,date
review_stars,,,,,,,,
1,356531,356531,356531,356531,356531,356531,356531,356531
2,169370,169370,169370,169370,169370,169370,169370,169370
3,193748,193748,193748,193748,193748,193748,193748,193748
4,373923,373923,373923,373923,373923,373923,373923,373923
5,591242,591242,591242,591242,591242,591242,591242,591242


In [49]:
df1_filtered = df[(df['useful']>1) & (df['review_stars']==1)]
df5_filtered = df[(df['useful']>2) & (df['review_stars']==5)]

In [37]:
df1_filtered.drop_duplicates(subset='text', keep='first',inplace=True)

/home/zhehao/anaconda3/envs/transformer/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [51]:
df5_filtered.drop_duplicates(subset='text', keep='first',inplace=True)

/home/zhehao/anaconda3/envs/transformer/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [38]:
df1_filtered.shape

(355914, 9)

In [50]:
df5_filtered.shape

(335812, 9)

### Seeing an example

In [59]:
df1_filtered.iloc[17]

'Worse than before. Overpriced, and HORRIBLE service. When a group says "We\'re really thirsty. Could you please bring water while we look at the menu?" You\'d better frickin\' BRING IT!  Our waiter was pushy, rude, and S...L...O...W... When we said that we weren\'t interested in hearing the specials, he read them anyway.  He constantly tried to upsell us. Every order placed was followed by a pause, and "Well, you know what would be really good...?" followed by a big description of a more expensive item or an add-on. The desperately needed water took almost twenty minutes to get to our table. The table next to us, who was seated at least five minutes after us, received their water, drinks, and first course before we even got our water.  Do I sound like I\'m obsessing about water? In the desert, dehydration is no joke, and it\'s indicative of the quality (bad - very very bad) of service that we received. The sushi was good, but not great. The main courses were simple but extremely expen

## Generate nonewline textfiles

In [26]:
# replace \n with space
df1_filtered = df1_filtered["text"].apply(lambda x : x.replace("\n\n", " "))


In [58]:
df1_filtered = df1_filtered.apply(lambda x : x.replace("\n", " "))

In [60]:
df1_filtered.to_csv('review_1_filtered_nonewline.txt', encoding='utf-8', header=None, index=None, sep=' ')


In [61]:
df1_filtered.to_csv('review_1_filtered_nonewline.csv', encoding='utf-8', index=None, sep=' ')

In [53]:
import csv

### Testing different formats (txt vs csv)

In [54]:
#csv
df1_filtered[['text']].to_csv("review_1_filtered.csv",index=False, quoting=csv.QUOTE_NONNUMERIC)
df5_filtered[['text']].to_csv("review_5_filtered.csv",index=False, quoting=csv.QUOTE_NONNUMERIC)

In [6]:
#text
df1_filtered[['text']].to_csv('review_1_filtered.txt', encoding='utf-8', header=None, index=None, sep=' ')
df5_filtered[['text']].to_csv('review_5_filtered.txt', encoding='utf-8', header=None, index=None, sep=' ')